## YOLO Dataset Conversion

Creates the complete structure for YOLO11 with:
- Properly structured directories (train/images, train/labels, etc.)
- TIFF images (4 channels) with unique names
- Annotations in YOLO format (.txt)
- dataset.yaml file with channel configuration

In [ ]:
import numpy as np
import json
import yaml
from PIL import Image
from tqdm import tqdm
import shutil
from pathlib import Path
import tifffile

BASE_DATA_DIR = Path(r"./data/thermalnumpy")
TIFF_DATA_DIR = Path(r"./data/tiff_4channel")
YOLO_OUTPUT_DIR = Path(r"./data/thermal_yolo")
TRAIN_JSON = Path(r"./data/Flug1_100-104Media_coco.json")
TEST_JSON = Path(r"./data/Flug1_105Media_coco.json")

In [ ]:
print("YOLO DATASET CREATION - Complete Structure")
print(f"\nOutput directory: {YOLO_OUTPUT_DIR}")
print(f"Image format: TIFF 4 channels (4, H, W)")
print(f"Annotation format: YOLO txt (class x_center y_center width height - normalized)")

# Create YOLO directory structure
train_images_dir = YOLO_OUTPUT_DIR / "train" / "images"
train_labels_dir = YOLO_OUTPUT_DIR / "train" / "labels"
val_images_dir = YOLO_OUTPUT_DIR / "val" / "images"
val_labels_dir = YOLO_OUTPUT_DIR / "val" / "labels"

for dir_path in [train_images_dir, train_labels_dir, val_images_dir, val_labels_dir]:
    dir_path.mkdir(parents=True, exist_ok=True)

print(f"\nDirectory structure created:")
print(f"   {train_images_dir}")
print(f"   {train_labels_dir}")
print(f"   {val_images_dir}")
print(f"   {val_labels_dir}")

def convert_coco_to_yolo_bbox(coco_bbox, img_width, img_height):
    """
    Convert COCO bbox [x, y, width, height] to YOLO format [x_center, y_center, width, height] normalized
    """
    x, y, w, h = coco_bbox
    x_center = x + w / 2
    y_center = y + h / 2
    x_center_norm = x_center / img_width
    y_center_norm = y_center / img_height
    w_norm = w / img_width
    h_norm = h / img_height
    return x_center_norm, y_center_norm, w_norm, h_norm

def convert_coco_segmentation_to_yolo(segmentation, img_width, img_height):
    """
    Convert COCO segmentation to YOLO format (normalized points)
    """
    if not segmentation or len(segmentation) == 0:
        return None
    polygon = segmentation[0] if isinstance(segmentation[0], list) else segmentation
    normalized_points = []
    for i in range(0, len(polygon), 2):
        x = polygon[i] / img_width
        y = polygon[i + 1] / img_height
        normalized_points.extend([x, y])
    return normalized_points

def process_yolo_split(input_dir, output_images_dir, output_labels_dir, json_path, split_name, category_mapping):
    """
    Process a split (train/val) for YOLO:
    - Copy/convert images with unique names (no subfolders)
    - Create .txt files with YOLO annotations
    """
    print(f"\n{'='*80}")
    print(f"Processing {split_name.upper()}")
    print(f"{'='*80}")
    print(f"JSON: {json_path}")
    if not json_path.exists():
        print(f"File not found: {json_path}")
        return 0, 0
    with open(json_path, 'r') as f:
        coco_data = json.load(f)
    images_info = {img['id']: img for img in coco_data.get('images', [])}
    annotations = coco_data.get('annotations', [])
    print(f"JSON stats:")
    print(f"   Images: {len(images_info)}")
    print(f"   Annotations: {len(annotations)}")
    annotations_by_image = {}
    for ann in annotations:
        img_id = ann['image_id']
        if img_id not in annotations_by_image:
            annotations_by_image[img_id] = []
        annotations_by_image[img_id].append(ann)
    success_count = 0
    error_count = 0
    for img_id, img_info in tqdm(images_info.items(), desc=f"Converting {split_name}"):
        try:
            file_name = img_info['file_name']
            img_width = img_info['width']
            img_height = img_info['height']
            unique_name = file_name.replace('/', '_').replace('\\', '_').replace('.npy', '.tiff')
            source_tiff = input_dir / file_name.replace('.npy', '.tiff')
            if not source_tiff.exists():
                source_numpy = BASE_DATA_DIR / split_name / "images" / file_name
                if source_numpy.exists():
                    print(f"\nTIFF not found, converting from numpy: {file_name}")
                    data = np.load(source_numpy)
                    data_4ch = data[:, :, :4]
                    data_transposed = np.transpose(data_4ch, (2, 0, 1))
                    dest_image = output_images_dir / unique_name
                    try:
                        import tifffile
                        tifffile.imwrite(dest_image, data_transposed, photometric='rgb')
                    except ImportError:
                        images_pil = [Image.fromarray(data_transposed[i]) for i in range(4)]
                        images_pil[0].save(dest_image, save_all=True, append_images=images_pil[1:], compression='tiff_deflate')
                else:
                    print(f"\nFile not found: {file_name}")
                    error_count += 1
                    continue
            else:
                dest_image = output_images_dir / unique_name
                shutil.copy2(source_tiff, dest_image)
            label_file = output_labels_dir / unique_name.replace('.tiff', '.txt')
            img_annotations = annotations_by_image.get(img_id, [])
            with open(label_file, 'w') as f:
                for ann in img_annotations:
                    category_id = ann['category_id']
                    class_id = category_mapping.get(category_id, category_id)
                    if 'segmentation' in ann and ann['segmentation']:
                        seg_points = convert_coco_segmentation_to_yolo(
                            ann['segmentation'], img_width, img_height
                        )
                        if seg_points:
                            points_str = ' '.join([f"{p:.6f}" for p in seg_points])
                            f.write(f"{class_id} {points_str}\n")
                    elif 'bbox' in ann:
                        x_c, y_c, w, h = convert_coco_to_yolo_bbox(
                            ann['bbox'], img_width, img_height
                        )
                        f.write(f"{class_id} {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}\n")
            success_count += 1
        except Exception as e:
            print(f"\nError with {img_info.get('file_name', 'unknown')}: {e}")
            error_count += 1
    print(f"\nConversion completed:")
    print(f"   Success: {success_count}")
    print(f"   Errors: {error_count}")
    return success_count, error_count

# Load categories from train JSON
print(f"\n{'='*80}")
print("CATEGORY ANALYSIS")
print(f"{'='*80}")
with open(TRAIN_JSON, 'r') as f:
    train_coco = json.load(f)
categories = train_coco.get('categories', [])
print(f"\nCategories found: {len(categories)}")
# Create mapping category_id → YOLO class_id (0-indexed)
category_mapping = {}
category_names = []
for i, cat in enumerate(sorted(categories, key=lambda x: x['id'])):
    category_mapping[cat['id']] = i
    category_names.append(cat['name'])
    print(f"   {cat['id']} → {i}: {cat['name']}")
# Process Train
train_input_tiff = TIFF_DATA_DIR / "train" / "images"
train_success, train_errors = process_yolo_split(
    train_input_tiff, train_images_dir, train_labels_dir, 
    TRAIN_JSON, "train", category_mapping
)
# Process Test (as validation in YOLO)
test_input_tiff = TIFF_DATA_DIR / "test" / "images"
val_success, val_errors = process_yolo_split(
    test_input_tiff, val_images_dir, val_labels_dir,
    TEST_JSON, "val", category_mapping
)
# Create dataset.yaml file
print(f"\n{'='*80}")
print("CREATING DATASET.YAML FILE")
print(f"{'='*80}")
yaml_content = {
    'path': str(YOLO_OUTPUT_DIR.absolute()),
    'train': 'train/images',
    'val': 'val/images',
    'test': 'val/images',  # Use val also for test
    'ch': 4,  # 4 channels (BGR + Thermal)
    'nc': len(category_names),
    'names': category_names
}
yaml_file = YOLO_OUTPUT_DIR / "dataset.yaml"
with open(yaml_file, 'w') as f:
    yaml.dump(yaml_content, f, default_flow_style=False, sort_keys=False)
print(f"\nYAML file created: {yaml_file}")
print(f"\nContent:")
print(f"   path: {yaml_content['path']}")
print(f"   train: {yaml_content['train']}")
print(f"   val: {yaml_content['val']}")
print(f"   ch: {yaml_content['ch']} (4 channels: BGR + Thermal)")
print(f"   nc: {yaml_content['nc']} (classes)")
print(f"   names: {yaml_content['names']}")
print(f"\n{'='*80}")
print("FINAL SUMMARY")
print(f"{'='*80}")
print(f"\nTRAIN:")
print(f"   Images converted: {train_success}")
print(f"   Errors: {train_errors}")
print(f"\nVAL:")
print(f"   Images converted: {val_success}")
print(f"   Errors: {val_errors}")
print(f"\nTotal: {train_success + val_success} images")
if train_success + val_success > 0:
    print(f"\nYOLO dataset created successfully!")
    print(f"\nFinal structure:")
    print(f"   {YOLO_OUTPUT_DIR}/")
    print(f"   ├── dataset.yaml")
    print(f"   ├── train/")
    print(f"   │   ├── images/  ({train_success} TIFF files)")
    print(f"   │   └── labels/  ({train_success} TXT files)")
    print(f"   └── val/")
    print(f"       ├── images/  ({val_success} TIFF files)")
    print(f"       └── labels/  ({val_success} TXT files)")
    print(f"\nConversion completed.")
else:
    print(f"\nError creating the dataset!")